> cnn_ensemble.py
* model architecture
* cnn training
* feature extraction for training + test
* training svm, random forest on training
* classification using heuristic ensembling

### imports

In [4]:
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from keras.layers import Dropout, LeakyReLU       
from keras.activations import relu, softmax, tanh, hard_sigmoid

import pydot, graphviz
from keras.utils.vis_utils import plot_model

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os, glob
from PIL import Image
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
import keras
from keras.callbacks import ModelCheckpoint

In [11]:
def hist_equ(img_path = '', show = False, img = None):
    if img is None:
        img = cv2.imread(data_path + img_path)

    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV) # RGB to YUB color space

    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0]) # histogram equalization in Y channel

    equ = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR) # inverse transform
    
    if show:    
        comp = np.hstack((img,equ)) # horizontal stack
        cv2.imwrite('original.bmp',img)
        cv2.imwrite('hist_equ.bmp',equ)
        cv2.imwrite('comp.bmp',comp)
        plt.imshow(comp)
        plt.show()
    return equ


In [12]:
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [4]:
def generate_segmented_images(_class,_id,data_split,flag_show):

    _class = str(_class)
    _id = str(_id)
    
    imgpath = 'PH2 Dataset images/'+'/IMD'+_id+'/IMD'+_id+'_Dermoscopic_Image/IMD'+_id+'.bmp'  
    maskpath = 'PH2 Dataset images/'+'/IMD'+_id+'/IMD'+_id+'_lesion/IMD'+_id+'_lesion.bmp'
    img2 = cv2.imread(imgpath)
    img = hist_equ(img = img2)
    img = adjust_gamma(img, 1.55)
    msk = cv2.imread(maskpath)
    
    #cv2.imshow('Main Image',img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    if flag_show:
        plt.imshow(img)
        plt.show()
        #print(img.shape)

        plt.imshow(msk)
        plt.show()


    msk = msk/255 # normalizing
    max_channels = np.amax([np.amax(msk[:,:,0]), np.amax(msk[:,:,1]), np.amax(msk[:,:,2])])
    
    msk = np.ndarray.astype(msk, dtype='uint8')
    
    gen_img = img*msk
    dirc = 'F:/Research important/Thesis/Active Phase/Part1 Scin Cancer/PH2Dataset experiments/classification/' + data_split
    dirc = os.path.join(dirc,_class)
    if not os.path.exists(dirc):
        os.makedirs(dirc)
        
    gen_img_name =  dirc + '/' + _id + '.png'
    cv2.imwrite(gen_img_name,gen_img)
    if flag_show:
        print(max_channels)
        plt.imshow(gen_img)
        plt.show()



### preparing data

In [5]:
import pandas
df = pandas.read_excel('PH2_dataset.xlsx') # The excel file was modified, unnecessary info was removed
#print the column names
print (df.columns)
#get the values for a given column
values = df['Image Name'].values
#get a data frame with selected columns
FORMAT = ['Image Name', 'Common Nevus', 'Atypical Nevus', 'Melanoma']
df_selected = df[FORMAT]
print(df_selected.loc[0][0])
print(df_selected.loc[0][1])
print(type(df_selected.loc[0][0]))
df_selected[1:4]

Index(['Image Name', 'Histological Diagnosis', 'Common Nevus',
       'Atypical Nevus', 'Melanoma', 'Asymmetry\n(0/1/2)',
       'Pigment Network\n(AT/T)', 'Dots/Globules\n(A/AT/T)', 'Streaks\n(A/P)',
       'Regression Areas\n(A/P)', 'Blue-Whitish Veil\n(A/P)', 'White', 'Red',
       'Light-Brown', 'Dark-Brown', 'Blue-Gray', 'Black'],
      dtype='object')
IMD003
X
<class 'str'>


,Image Name,Common Nevus,Atypical Nevus,Melanoma
1,IMD009,X,NaN,NaN
2,IMD016,X,NaN,NaN
3,IMD022,X,NaN,NaN


In [6]:
df_selected.loc[0][0][-3:]

'003'

In [7]:
num_samples = 200
class0 = []
class1 = []
class2 = []
for ns in range(num_samples):
    img_id = df_selected.loc[ns][0][-3:]
    com_nev = df_selected.loc[ns][1]
    atyp_nev = df_selected.loc[ns][2]
    melan = df_selected.loc[ns][3]
    _class = -1
    if com_nev == 'X':
        _class = 0
        class0.append(img_id)
    elif atyp_nev == 'X':
        _class = 1
        class1.append(img_id)
    elif melan == 'X':
        _class = 2
        class2.append(img_id)
    #print(_class)
    #generate_segmented_images(_class,img_id,flag_show = False)
    

In [8]:
import random
random.seed(1997) # to reproduce results

train_split = 0.85
random.shuffle(class0)
random.shuffle(class1)
random.shuffle(class2)

train_0 = class0[0:int(len(class0)*train_split)]
test_0 = class0[int(len(class0)*train_split):]
print(train_0)
print(test_0)

train_1 = class1[0:int(len(class1)*train_split)]
test_1 = class1[int(len(class1)*train_split):]
print(train_1)
print(test_1)

train_2 = class2[0:int(len(class2)*train_split)]
test_2 = class2[int(len(class2)*train_split):]
print(train_2)
print(test_2)

['045', '156', '044', '372', '146', '147', '105', '125', '379', '381', '025', '020', '107', '198', '204', '134', '159', '150', '203', '374', '010', '050', '390', '200', '160', '389', '385', '173', '003', '009', '152', '400', '176', '197', '112', '402', '035', '364', '132', '142', '384', '133', '392', '367', '182', '375', '395', '394', '092', '371', '016', '039', '206', '022', '041', '397', '380', '135', '143', '103', '365', '207', '378', '101', '383', '024', '144', '017']
['199', '038', '162', '177', '208', '108', '196', '399', '118', '161', '042', '175']
['032', '149', '437', '434', '398', '393', '076', '386', '037', '251', '036', '049', '370', '210', '433', '431', '356', '328', '013', '388', '171', '002', '040', '164', '153', '256', '278', '120', '436', '008', '157', '043', '057', '304', '139', '368', '018', '279', '004', '033', '312', '369', '031', '047', '019', '280', '138', '305', '427', '030', '396', '166', '169', '155', '126', '027', '140', '339', '430', '226', '306', '014', '02

In [9]:
print(len(class0))
print(len(train_0) + len(test_0))

print(len(class1))
print(len(train_1) + len(test_1))

print(len(class2))
print(len(train_2) + len(test_2))

80
80
80
80
40
40


In [13]:
for i in range(len(train_0)):
    generate_segmented_images(0,train_0[i],'train',flag_show = False)

for i in range(len(test_0)):
    generate_segmented_images(0,test_0[i],'test',flag_show = False)
    
for i in range(len(train_1)):
    generate_segmented_images(1,train_1[i],'train',flag_show = False)

for i in range(len(test_1)):
    generate_segmented_images(1,test_1[i],'test',flag_show = False)
    
    
for i in range(len(train_2)):
    generate_segmented_images(2,train_2[i],'train',flag_show = False)

for i in range(len(test_2)):
    generate_segmented_images(2,test_2[i],'test',flag_show = False)

### model architecture

In [14]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3),   # kernel size :
                 input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) # strides up :    strides down :
model.add(BatchNormalization()) # BN on :      BN off : 
model.add(LeakyReLU(alpha=0.1)) # sigmoid :      relu :      tanh : 

# LeakyReLU(alpha=0.1)
# model.add(Dropout(0.5))

model.add(Conv2D(32, (5, 5)))     # kernel size :
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization()) # BN on :      BN off : 
model.add(LeakyReLU(alpha=0.1)) # sigmoid :      relu :      tanh :
#model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 221, 221, 16)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 221, 221, 16)      64        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 221, 221, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 217, 217, 32)      12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 108, 108, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 108, 108, 32)      128       
__________

In [14]:
TRAINING_DIR = 'train'
IMAGE_SIZE = 224
BATCH_SIZE = 4

In [17]:
model.compile(loss=categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum = 0.9, nesterov = True),
              metrics=['accuracy'])
# checkpoint
path = "best_weight.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]



data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")




Instructions for updating:
Use the retry module or similar alternatives.
Found 145 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


### tensorboard
> tensorboard --logdir ./Graph

> htttp://localhost:6006

In [19]:
# 80, 80, 40
c_weight = {0:1, 1:1, 2:2}

model.fit_generator(train_generator,
                       steps_per_epoch=50,
                       validation_data=validation_generator, 
                       epochs=10, 
                       verbose=1,
                       class_weight = c_weight,
                       callbacks = callbacks_list,
                       validation_steps=10,
                       initial_epoch=0)

Epoch 1/10
50/50 [==============================] - 152s 3s/step - loss: 1.0943 - acc: 0.5203 - val_loss: 0.7161 - val_acc: 0.5946

Epoch 00001: val_acc improved from 0.56757 to 0.59459, saving model to best_weight.hdf5
Epoch 2/10
50/50 [==============================] - 158s 3s/step - loss: 1.0757 - acc: 0.5349 - val_loss: 0.6813 - val_acc: 0.7297

Epoch 00002: val_acc improved from 0.59459 to 0.72973, saving model to best_weight.hdf5
Epoch 3/10
50/50 [==============================] - 165s 3s/step - loss: 1.0519 - acc: 0.5302 - val_loss: 0.7100 - val_acc: 0.6486

Epoch 00003: val_acc did not improve
Epoch 4/10
50/50 [==============================] - 152s 3s/step - loss: 1.0066 - acc: 0.5253 - val_loss: 0.7110 - val_acc: 0.5946

Epoch 00004: val_acc did not improve
Epoch 5/10
50/50 [==============================] - 146s 3s/step - loss: 0.9445 - acc: 0.5699 - val_loss: 0.8151 - val_acc: 0.5405

Epoch 00005: val_acc did not improve
Epoch 6/10
50/50 [==============================] - 1

In [21]:
## model 2
model2 = Sequential()

model2.add(Conv2D(16, kernel_size=(3, 3),   # kernel size :
                 input_shape=(224,224,3)))
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) # strides up :    strides down :
model2.add(BatchNormalization()) # BN on :      BN off : 
model2.add(Activation('relu')) # sigmoid :      relu :      tanh : 

# LeakyReLU(alpha=0.1)
# model.add(Dropout(0.5))

model2.add(Conv2D(32, (3, 3)))     # kernel size :
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(BatchNormalization()) # BN on :      BN off : 
model2.add(Activation('relu')) # sigmoid :      relu :      tanh :
#model.add(Dropout(0.5))


model2.add(Flatten())
model2.add(Dense(128))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.2))

model2.add(Dense(3))
model2.add(BatchNormalization())
model2.add(Activation('softmax'))

In [22]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 221, 221, 16)      0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 221, 221, 16)      64        
_________________________________________________________________
activation_2 (Activation)    (None, 221, 221, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 219, 219, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 109, 109, 32)      0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 109, 109, 32)      128       
__________

In [23]:
model2.compile(loss=categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum = 0.9, nesterov = True),
              metrics=['accuracy'])
# checkpoint
path = "best_weight2.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]



data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")




Found 145 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [24]:
# 80, 80, 40
c_weight = {0:1, 1:1, 2:2}

model2.fit_generator(train_generator,
                       steps_per_epoch=50,
                       validation_data=validation_generator, 
                       epochs=10, 
                       verbose=1,
                       class_weight = c_weight,
                       callbacks = callbacks_list,
                       validation_steps=10,
                       initial_epoch=0)

Epoch 1/10
50/50 [==============================] - 156s 3s/step - loss: 1.2870 - acc: 0.3902 - val_loss: 0.9559 - val_acc: 0.4865

Epoch 00001: val_acc improved from -inf to 0.48649, saving model to best_weight2.hdf5
Epoch 2/10
50/50 [==============================] - 136s 3s/step - loss: 1.0451 - acc: 0.4851 - val_loss: 0.8594 - val_acc: 0.2973

Epoch 00002: val_acc did not improve
Epoch 3/10
50/50 [==============================] - 144s 3s/step - loss: 1.0871 - acc: 0.5252 - val_loss: 0.7211 - val_acc: 0.5946

Epoch 00003: val_acc improved from 0.48649 to 0.59459, saving model to best_weight2.hdf5
Epoch 4/10
50/50 [==============================] - 131s 3s/step - loss: 0.9484 - acc: 0.6004 - val_loss: 0.8611 - val_acc: 0.4865

Epoch 00004: val_acc did not improve
Epoch 5/10
50/50 [==============================] - 142s 3s/step - loss: 0.8213 - acc: 0.6849 - val_loss: 0.7041 - val_acc: 0.7568

Epoch 00005: val_acc improved from 0.59459 to 0.75676, saving model to best_weight2.hdf5
Ep

In [25]:
## model 3
model3 = Sequential()

model3.add(Conv2D(16, kernel_size=(3, 3),   # kernel size :
                 input_shape=(224,224,3)))
model3.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) # strides up :    strides down :
#model3.add(BatchNormalization()) # BN on :      BN off : 
model3.add(Activation('relu')) # sigmoid :      relu :      tanh : 

# LeakyReLU(alpha=0.1)
# model.add(Dropout(0.5))

model3.add(Conv2D(32, (3, 3)))     # kernel size :
model3.add(MaxPooling2D(pool_size=(2, 2)))

#model3.add(BatchNormalization()) # BN on :      BN off : 
model3.add(Activation('relu')) # sigmoid :      relu :      tanh :
#model.add(Dropout(0.5))


model3.add(Flatten())
model3.add(Dense(128))
#model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(Dropout(0.25))

model3.add(Dense(3))
#model2.add(BatchNormalization())
model3.add(Activation('softmax'))

In [26]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 221, 221, 16)      0         
_________________________________________________________________
activation_6 (Activation)    (None, 221, 221, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 219, 219, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 109, 109, 32)      0         
_________________________________________________________________
activation_7 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 380192)            0         
__________

In [27]:
optm = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model3.compile(loss=categorical_crossentropy,
              optimizer=optm,
              metrics=['accuracy'])
# checkpoint
path = "best_weight3.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]



data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")




Found 145 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [28]:
# 80, 80, 40
c_weight = {0:1, 1:1, 2:2}

model3.fit_generator(train_generator,
                       steps_per_epoch=50,
                       validation_data=validation_generator, 
                       epochs=10, 
                       verbose=1,
                       class_weight = c_weight,
                       callbacks = callbacks_list,
                       validation_steps=10,
                       initial_epoch=0)

Epoch 1/10
 2/50 [>.............................] - ETA: 8:31 - loss: 8.6502 - acc: 0.1250 

C:\Anaconda3\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (3.935483). Check your callbacks.
  % delta_t_median)


50/50 [==============================] - 294s 6s/step - loss: 5.8614 - acc: 0.4348 - val_loss: 0.9407 - val_acc: 0.5676

Epoch 00001: val_acc improved from -inf to 0.56757, saving model to best_weight3.hdf5
Epoch 2/10
50/50 [==============================] - 103s 2s/step - loss: 1.2575 - acc: 0.5951 - val_loss: 0.8725 - val_acc: 0.5405

Epoch 00002: val_acc did not improve
Epoch 3/10
50/50 [==============================] - 102s 2s/step - loss: 0.3148 - acc: 0.9100 - val_loss: 0.8215 - val_acc: 0.5405

Epoch 00003: val_acc did not improve
Epoch 4/10
50/50 [==============================] - 118s 2s/step - loss: 0.0275 - acc: 1.0000 - val_loss: 1.1000 - val_acc: 0.4595

Epoch 00004: val_acc did not improve
Epoch 5/10
50/50 [==============================] - 126s 3s/step - loss: 0.0524 - acc: 0.9850 - val_loss: 1.4301 - val_acc: 0.5405

Epoch 00005: val_acc did not improve
Epoch 6/10
50/50 [==============================] - 124s 2s/step - loss: 0.0121 - acc: 0.9950 - val_loss: 1.3986 - va

In [29]:
## model 4
model4 = Sequential()

model4.add(Conv2D(16, kernel_size=(3, 3),   # kernel size :
                 input_shape=(224,224,3)))
model4.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) # strides up :    strides down :
#model3.add(BatchNormalization()) # BN on :      BN off : 
model4.add(Activation('relu')) # sigmoid :      relu :      tanh : 

# LeakyReLU(alpha=0.1)
# model.add(Dropout(0.1))

model4.add(Conv2D(32, (3, 3)))     # kernel size :
model4.add(MaxPooling2D(pool_size=(2, 2)))

#model3.add(BatchNormalization()) # BN on :      BN off : 
model4.add(Activation('relu')) # sigmoid :      relu :      tanh :
# model.add(Dropout(0.1))


model4.add(Flatten())
model4.add(Dense(128))
#model3.add(BatchNormalization())
model4.add(Activation('sigmoid'))
model4.add(Dropout(0.5))

model4.add(Dense(3))
#model2.add(BatchNormalization())
model4.add(Activation('softmax'))

In [30]:
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 221, 221, 16)      0         
_________________________________________________________________
activation_10 (Activation)   (None, 221, 221, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 219, 219, 32)      4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 109, 109, 32)      0         
_________________________________________________________________
activation_11 (Activation)   (None, 109, 109, 32)      0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 380192)            0         
__________

In [33]:
optm = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999, epsilon=None, decay=0.0, amsgrad=False)
model4.compile(loss=categorical_crossentropy,
              optimizer=optm,
              metrics=['accuracy'])
# checkpoint
path = "best_weight4.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]



data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")




Found 145 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [34]:
# 80, 80, 40
c_weight = {0:1, 1:1, 2:2}

model4.fit_generator(train_generator,
                       steps_per_epoch=50,
                       validation_data=validation_generator, 
                       epochs=10, 
                       verbose=1,
                       class_weight = c_weight,
                       callbacks = callbacks_list,
                       validation_steps=10,
                       initial_epoch=0)

Epoch 1/10
50/50 [==============================] - 150s 3s/step - loss: 1.5557 - acc: 0.3999 - val_loss: 1.0864 - val_acc: 0.4865

Epoch 00001: val_acc improved from -inf to 0.48649, saving model to best_weight4.hdf5
Epoch 2/10
50/50 [==============================] - 123s 2s/step - loss: 1.4427 - acc: 0.3850 - val_loss: 1.0994 - val_acc: 0.3784

Epoch 00002: val_acc did not improve
Epoch 3/10
50/50 [==============================] - 123s 2s/step - loss: 1.6505 - acc: 0.3252 - val_loss: 1.0872 - val_acc: 0.3243

Epoch 00003: val_acc did not improve
Epoch 4/10
50/50 [==============================] - 122s 2s/step - loss: 1.5165 - acc: 0.3749 - val_loss: 0.9820 - val_acc: 0.4324

Epoch 00004: val_acc did not improve
Epoch 5/10
50/50 [==============================] - 133s 3s/step - loss: 1.3585 - acc: 0.5202 - val_loss: 0.8356 - val_acc: 0.5676

Epoch 00005: val_acc improved from 0.48649 to 0.56757, saving model to best_weight4.hdf5
Epoch 6/10
50/50 [==============================] - 12

In [37]:
## model 5
model5 = Sequential()

model5.add(Conv2D(16, kernel_size=(3, 3),   # kernel size :
                 input_shape=(224,224,3)))
#model5.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) # strides up :    strides down :
#model3.add(BatchNormalization()) # BN on :      BN off : 
model5.add(Activation('relu')) # sigmoid :      relu :      tanh : 

# LeakyReLU(alpha=0.1)
model5.add(Dropout(0.1))

model5.add(Conv2D(32, (3, 3)))     # kernel size :
model5.add(MaxPooling2D(pool_size=(2, 2)))

#model3.add(BatchNormalization()) # BN on :      BN off : 
model5.add(Activation('relu')) # sigmoid :      relu :      tanh :
model5.add(Dropout(0.1))

model5.add(Conv2D(64, (3, 3)))     # kernel size :
model5.add(MaxPooling2D(pool_size=(2, 2)))

#model3.add(BatchNormalization()) # BN on :      BN off : 
model5.add(Activation('relu')) # sigmoid :      relu :      tanh :
model5.add(Dropout(0.1))


model5.add(Flatten())
model5.add(Dense(64))
#model3.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(Dropout(0.4))


model5.add(Dense(128))
#model3.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(Dropout(0.4))

model5.add(Dense(3))
#model2.add(BatchNormalization())
model5.add(Activation('softmax'))

In [38]:
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 222, 222, 16)      448       
_________________________________________________________________
activation_23 (Activation)   (None, 222, 222, 16)      0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 222, 222, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 220, 220, 32)      4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 110, 110, 32)      0         
_________________________________________________________________
activation_24 (Activation)   (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 110, 110, 32)      0         
__________

In [41]:
optm = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999, epsilon=None, decay=0.0, amsgrad=False)
model5.compile(loss=categorical_crossentropy,
              optimizer=optm,
              metrics=['accuracy'])
# checkpoint
path = "best_weight5.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]



data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE*2, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE*2, subset="validation")




Found 145 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [42]:
# 80, 80, 40
c_weight = {0:1, 1:1, 2:2}

model5.fit_generator(train_generator,
                       steps_per_epoch=50,
                       validation_data=validation_generator, 
                       epochs=10, 
                       verbose=1,
                       class_weight = c_weight,
                       callbacks = callbacks_list,
                       validation_steps=10,
                       initial_epoch=0)

Epoch 1/10
50/50 [==============================] - 118s 2s/step - loss: 1.2389 - acc: 0.2553 - val_loss: 0.9896 - val_acc: 0.5909

Epoch 00001: val_acc improved from -inf to 0.59091, saving model to best_weight5.hdf5
Epoch 2/10
50/50 [==============================] - 125s 2s/step - loss: 1.0243 - acc: 0.5545 - val_loss: 0.7764 - val_acc: 0.5606

Epoch 00002: val_acc did not improve
Epoch 3/10
50/50 [==============================] - 95s 2s/step - loss: 0.7134 - acc: 0.6523 - val_loss: 1.1981 - val_acc: 0.5000

Epoch 00003: val_acc did not improve
Epoch 4/10
50/50 [==============================] - 95s 2s/step - loss: 0.7397 - acc: 0.6876 - val_loss: 0.9024 - val_acc: 0.5303

Epoch 00004: val_acc did not improve
Epoch 5/10
50/50 [==============================] - 92s 2s/step - loss: 0.4160 - acc: 0.8098 - val_loss: 1.1161 - val_acc: 0.6970

Epoch 00005: val_acc improved from 0.59091 to 0.69697, saving model to best_weight5.hdf5
Epoch 6/10
50/50 [==============================] - 97s 2

In [46]:
for i, layer in enumerate(model5.layers):
    print(i, layer.name)

0 conv2d_15
1 activation_23
2 dropout_15
3 conv2d_16
4 max_pooling2d_13
5 activation_24
6 dropout_16
7 conv2d_17
8 max_pooling2d_14
9 activation_25
10 dropout_17
11 flatten_8
12 dense_12
13 activation_26
14 dropout_18
15 dense_13
16 activation_27
17 dropout_19
18 dense_14
19 activation_28


In [53]:
# Extract features from an arbitrary intermediate layer
from keras.models import Model
from keras.preprocessing import image
model_fe = Model(inputs=model5.input, outputs=model5.get_layer('dense_13').output)

# load an image and preprocess it
img_path = 'test.bmp'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# get the features 
features = model_fe.predict(x)

In [55]:
print(features)

[[ -258.47922    -168.22876      32.303875    628.96124     298.69937
   -190.73158    -369.96774      86.12593      54.811954    134.03952
   -128.10455     497.63858     804.23627    -313.0369     -820.4245
    279.30426    1205.5005      324.50034     -16.94054    -661.7363
    497.626      -486.84073     173.10646    -479.46912    -401.12036
    504.56284     336.819       150.16965    -433.0042        7.2921257
    428.5451       -1.2626623   203.04216     899.2101     -442.60178
     16.11631     552.1154     -612.38477    -328.64893    -350.64658
   -463.8479     -213.48932    1206.1561       10.02821     803.36426
    -17.090204   -740.31384    -751.6819     -144.64534     686.55676
   -336.5051     -787.98816     405.46695     657.82855      12.142002
    146.71025    -555.3851      493.86185      86.37762     -17.741146
   -493.24414    -323.51         32.564068   -831.3529      194.96825
   -306.36255     262.69498     539.1898      320.4866      269.64627
   -276.20078     

In [1]:
print(2)

2


In [6]:
import Augmentor
p = Augmentor.Pipeline("train")
p.random_distortion(probability = 1, grid_width = 3, grid_height = 3, magnitude = 10)
p.flip_random(probability = 1)
p.rotate_without_crop(probability=1, max_left_rotation=360, max_right_rotation=360, expand=False)
p.skew(probability=1, magnitude=0.8)
p.sample(2720)

Initialised with 170 image(s) found.
Output directory set to train\output.

Processing <PIL.Image.Image image mode=RGB size=764x576 at 0xC02F696550>: 100%|█| 2720/2720 [08:48<00:00,  3.73 Samples/s]


In [11]:
## model 6
model6 = Sequential()

model6.add(Conv2D(16, kernel_size=(3, 3),   # kernel size :
                 input_shape=(224,224,3)))
model6.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) # strides up :    strides down :
#model3.add(BatchNormalization()) # BN on :      BN off : 
model6.add(Activation('relu')) # sigmoid :      relu :      tanh : 

# LeakyReLU(alpha=0.1)
model6.add(Dropout(0.1))

model6.add(Conv2D(32, (3, 3)))     # kernel size :
model6.add(MaxPooling2D(pool_size=(2, 2)))

#model3.add(BatchNormalization()) # BN on :      BN off : 
model6.add(Activation('relu')) # sigmoid :      relu :      tanh :
model6.add(Dropout(0.1))

model6.add(Conv2D(64, (3, 3)))     # kernel size :
model6.add(MaxPooling2D(pool_size=(2, 2)))

#model3.add(BatchNormalization()) # BN on :      BN off : 
model6.add(Activation('relu')) # sigmoid :      relu :      tanh :
model6.add(Dropout(0.2))


model6.add(Flatten())
model6.add(Dense(64))
#model3.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(Dropout(0.4))


model6.add(Dense(128))
#model3.add(BatchNormalization())
model6.add(Activation('relu'))
model6.add(Dropout(0.5))

model6.add(Dense(3))
#model2.add(BatchNormalization())
model6.add(Activation('softmax'))

In [13]:
model6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 221, 221, 16)      0         
_________________________________________________________________
activation_6 (Activation)    (None, 221, 221, 16)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 221, 221, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 219, 219, 32)      4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 109, 109, 32)      0         
_________________________________________________________________
activation_7 (Activation)    (None, 109, 109, 32)      0         
__________

In [16]:
optm = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999, epsilon=None, decay=0.0, amsgrad=True)
model6.compile(loss=categorical_crossentropy,
              optimizer=optm,
              metrics=['accuracy'])
# checkpoint
path = "best_weight6.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]



data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE*2, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=1997,
                                                     class_mode='categorical', batch_size=BATCH_SIZE*2, subset="validation")




Instructions for updating:
Use the retry module or similar alternatives.
Found 2457 images belonging to 3 classes.
Found 433 images belonging to 3 classes.


In [20]:
# 80, 80, 40
c_weight = {0:1, 1:1, 2:2}

model6.fit_generator(train_generator,
                       steps_per_epoch=2457//8,
                       validation_data=validation_generator, 
                       epochs=10, 
                       verbose=1,
                       class_weight = c_weight,
                       callbacks = callbacks_list,
                       validation_steps=433//8,
                       initial_epoch=0)

Epoch 1/10
204/307 [==================>...........] - ETA: 12:45 - loss: 1.1022 - acc: 0.50 - ETA: 11:54 - loss: 0.8405 - acc: 0.62 - ETA: 11:25 - loss: 0.8849 - acc: 0.58 - ETA: 11:05 - loss: 0.8804 - acc: 0.56 - ETA: 10:49 - loss: 0.8927 - acc: 0.55 - ETA: 10:49 - loss: 0.8597 - acc: 0.54 - ETA: 10:48 - loss: 0.8753 - acc: 0.53 - ETA: 10:46 - loss: 0.9555 - acc: 0.53 - ETA: 10:40 - loss: 0.9392 - acc: 0.52 - ETA: 10:34 - loss: 0.9486 - acc: 0.52 - ETA: 10:28 - loss: 0.9638 - acc: 0.48 - ETA: 10:23 - loss: 0.9799 - acc: 0.50 - ETA: 10:17 - loss: 1.0533 - acc: 0.48 - ETA: 10:13 - loss: 1.0739 - acc: 0.47 - ETA: 10:11 - loss: 1.0464 - acc: 0.48 - ETA: 10:07 - loss: 1.0485 - acc: 0.48 - ETA: 10:05 - loss: 1.0305 - acc: 0.50 - ETA: 10:02 - loss: 1.0344 - acc: 0.51 - ETA: 9:59 - loss: 1.0519 - acc: 0.5000 - ETA: 9:56 - loss: 1.0382 - acc: 0.506 - ETA: 9:52 - loss: 1.0326 - acc: 0.494 - ETA: 9:49 - loss: 1.0358 - acc: 0.494 - ETA: 9:46 - loss: 1.0322 - acc: 0.483 - ETA: 9:44 - loss: 1.0349 

204/307 [==================>...........] - ETA: 13:11 - loss: 0.5959 - acc: 0.75 - ETA: 20:03 - loss: 0.6000 - acc: 0.75 - ETA: 20:40 - loss: 0.6317 - acc: 0.70 - ETA: 19:37 - loss: 0.7203 - acc: 0.65 - ETA: 18:00 - loss: 0.7497 - acc: 0.62 - ETA: 16:51 - loss: 0.8045 - acc: 0.58 - ETA: 21:47 - loss: 0.8085 - acc: 0.58 - ETA: 20:27 - loss: 0.8135 - acc: 0.54 - ETA: 19:42 - loss: 0.8132 - acc: 0.52 - ETA: 19:54 - loss: 0.8407 - acc: 0.52 - ETA: 19:31 - loss: 0.8268 - acc: 0.53 - ETA: 18:57 - loss: 0.8502 - acc: 0.52 - ETA: 18:34 - loss: 0.8621 - acc: 0.51 - ETA: 18:12 - loss: 0.8694 - acc: 0.53 - ETA: 17:50 - loss: 0.8679 - acc: 0.52 - ETA: 17:26 - loss: 0.8646 - acc: 0.53 - ETA: 17:01 - loss: 0.8627 - acc: 0.52 - ETA: 16:36 - loss: 0.8529 - acc: 0.54 - ETA: 16:12 - loss: 0.8705 - acc: 0.52 - ETA: 15:51 - loss: 0.8641 - acc: 0.52 - ETA: 15:31 - loss: 0.8633 - acc: 0.51 - ETA: 15:17 - loss: 0.8563 - acc: 0.51 - ETA: 14:59 - loss: 0.8580 - acc: 0.52 - ETA: 14:47 - loss: 0.8644 - acc: 0.52

307/307 [==============================] - ETA: 3:52 - loss: 0.8311 - acc: 0.583 - ETA: 3:50 - loss: 0.8328 - acc: 0.582 - ETA: 3:47 - loss: 0.8324 - acc: 0.582 - ETA: 3:45 - loss: 0.8340 - acc: 0.581 - ETA: 3:43 - loss: 0.8324 - acc: 0.582 - ETA: 3:40 - loss: 0.8353 - acc: 0.582 - ETA: 3:38 - loss: 0.8352 - acc: 0.581 - ETA: 3:36 - loss: 0.8364 - acc: 0.579 - ETA: 3:33 - loss: 0.8378 - acc: 0.579 - ETA: 3:31 - loss: 0.8382 - acc: 0.578 - ETA: 3:29 - loss: 0.8397 - acc: 0.577 - ETA: 3:26 - loss: 0.8378 - acc: 0.579 - ETA: 3:24 - loss: 0.8370 - acc: 0.579 - ETA: 3:22 - loss: 0.8392 - acc: 0.578 - ETA: 3:20 - loss: 0.8393 - acc: 0.577 - ETA: 3:17 - loss: 0.8382 - acc: 0.577 - ETA: 3:15 - loss: 0.8385 - acc: 0.576 - ETA: 3:13 - loss: 0.8400 - acc: 0.576 - ETA: 3:10 - loss: 0.8396 - acc: 0.575 - ETA: 3:08 - loss: 0.8412 - acc: 0.574 - ETA: 3:06 - loss: 0.8410 - acc: 0.574 - ETA: 3:03 - loss: 0.8420 - acc: 0.574 - ETA: 3:01 - loss: 0.8422 - acc: 0.572 - ETA: 2:59 - loss: 0.8434 - acc: 0.572

204/307 [==================>...........] - ETA: 12:43 - loss: 0.6942 - acc: 0.62 - ETA: 12:06 - loss: 0.7436 - acc: 0.68 - ETA: 12:22 - loss: 0.7620 - acc: 0.66 - ETA: 12:34 - loss: 0.8401 - acc: 0.59 - ETA: 12:18 - loss: 0.7958 - acc: 0.60 - ETA: 12:05 - loss: 0.7906 - acc: 0.56 - ETA: 11:53 - loss: 0.7419 - acc: 0.60 - ETA: 11:55 - loss: 0.7217 - acc: 0.60 - ETA: 11:52 - loss: 0.7369 - acc: 0.59 - ETA: 11:52 - loss: 0.7716 - acc: 0.60 - ETA: 11:42 - loss: 0.7744 - acc: 0.57 - ETA: 11:40 - loss: 0.7526 - acc: 0.58 - ETA: 11:43 - loss: 0.7892 - acc: 0.57 - ETA: 11:46 - loss: 0.7816 - acc: 0.57 - ETA: 12:04 - loss: 0.8034 - acc: 0.57 - ETA: 12:01 - loss: 0.7928 - acc: 0.57 - ETA: 11:55 - loss: 0.7797 - acc: 0.56 - ETA: 11:51 - loss: 0.7770 - acc: 0.56 - ETA: 11:48 - loss: 0.7667 - acc: 0.57 - ETA: 11:41 - loss: 0.7626 - acc: 0.57 - ETA: 11:36 - loss: 0.7636 - acc: 0.57 - ETA: 11:31 - loss: 0.7468 - acc: 0.58 - ETA: 11:26 - loss: 0.7376 - acc: 0.58 - ETA: 11:19 - loss: 0.7331 - acc: 0.59

307/307 [==============================] - ETA: 3:53 - loss: 0.7719 - acc: 0.597 - ETA: 3:51 - loss: 0.7718 - acc: 0.596 - ETA: 3:48 - loss: 0.7704 - acc: 0.597 - ETA: 3:46 - loss: 0.7702 - acc: 0.597 - ETA: 3:43 - loss: 0.7703 - acc: 0.597 - ETA: 3:41 - loss: 0.7696 - acc: 0.598 - ETA: 3:39 - loss: 0.7684 - acc: 0.598 - ETA: 3:36 - loss: 0.7679 - acc: 0.599 - ETA: 3:34 - loss: 0.7720 - acc: 0.599 - ETA: 3:31 - loss: 0.7724 - acc: 0.598 - ETA: 3:29 - loss: 0.7719 - acc: 0.598 - ETA: 3:27 - loss: 0.7718 - acc: 0.597 - ETA: 3:24 - loss: 0.7693 - acc: 0.598 - ETA: 3:22 - loss: 0.7689 - acc: 0.598 - ETA: 3:20 - loss: 0.7684 - acc: 0.598 - ETA: 3:18 - loss: 0.7676 - acc: 0.598 - ETA: 3:16 - loss: 0.7675 - acc: 0.597 - ETA: 3:13 - loss: 0.7661 - acc: 0.598 - ETA: 3:11 - loss: 0.7660 - acc: 0.598 - ETA: 3:09 - loss: 0.7653 - acc: 0.598 - ETA: 3:06 - loss: 0.7653 - acc: 0.597 - ETA: 3:04 - loss: 0.7650 - acc: 0.597 - ETA: 3:02 - loss: 0.7648 - acc: 0.598 - ETA: 3:00 - loss: 0.7653 - acc: 0.597

...

307/307 [==============================] - ETA: 3:27 - loss: 0.6423 - acc: 0.667 - ETA: 3:25 - loss: 0.6412 - acc: 0.667 - ETA: 3:23 - loss: 0.6415 - acc: 0.667 - ETA: 3:21 - loss: 0.6407 - acc: 0.667 - ETA: 3:19 - loss: 0.6416 - acc: 0.667 - ETA: 3:17 - loss: 0.6399 - acc: 0.669 - ETA: 3:15 - loss: 0.6433 - acc: 0.668 - ETA: 3:13 - loss: 0.6433 - acc: 0.668 - ETA: 3:11 - loss: 0.6436 - acc: 0.669 - ETA: 3:09 - loss: 0.6425 - acc: 0.669 - ETA: 3:07 - loss: 0.6420 - acc: 0.669 - ETA: 3:05 - loss: 0.6415 - acc: 0.670 - ETA: 3:03 - loss: 0.6418 - acc: 0.668 - ETA: 3:01 - loss: 0.6409 - acc: 0.668 - ETA: 2:59 - loss: 0.6417 - acc: 0.668 - ETA: 2:57 - loss: 0.6418 - acc: 0.668 - ETA: 2:55 - loss: 0.6406 - acc: 0.669 - ETA: 2:52 - loss: 0.6407 - acc: 0.670 - ETA: 2:50 - loss: 0.6405 - acc: 0.669 - ETA: 2:48 - loss: 0.6388 - acc: 0.671 - ETA: 2:46 - loss: 0.6391 - acc: 0.670 - ETA: 2:44 - loss: 0.6391 - acc: 0.669 - ETA: 2:42 - loss: 0.6390 - acc: 0.671 - ETA: 2:40 - loss: 0.6387 - acc: 0.670

204/307 [==================>...........] - ETA: 10:07 - loss: 0.3868 - acc: 0.87 - ETA: 10:05 - loss: 0.4443 - acc: 0.87 - ETA: 10:01 - loss: 0.5882 - acc: 0.79 - ETA: 10:02 - loss: 0.5827 - acc: 0.78 - ETA: 10:03 - loss: 0.5603 - acc: 0.75 - ETA: 9:58 - loss: 0.5394 - acc: 0.7917 - ETA: 10:02 - loss: 0.5434 - acc: 0.78 - ETA: 10:05 - loss: 0.5025 - acc: 0.81 - ETA: 10:02 - loss: 0.5489 - acc: 0.80 - ETA: 9:58 - loss: 0.5110 - acc: 0.8250 - ETA: 9:55 - loss: 0.5463 - acc: 0.818 - ETA: 9:53 - loss: 0.5241 - acc: 0.822 - ETA: 9:50 - loss: 0.5230 - acc: 0.817 - ETA: 9:48 - loss: 0.5229 - acc: 0.812 - ETA: 9:45 - loss: 0.5080 - acc: 0.808 - ETA: 9:43 - loss: 0.4974 - acc: 0.812 - ETA: 9:40 - loss: 0.4939 - acc: 0.816 - ETA: 9:16 - loss: 0.5393 - acc: 0.770 - ETA: 9:15 - loss: 0.5299 - acc: 0.769 - ETA: 9:14 - loss: 0.5144 - acc: 0.781 - ETA: 9:13 - loss: 0.5147 - acc: 0.773 - ETA: 9:11 - loss: 0.5262 - acc: 0.767 - ETA: 9:10 - loss: 0.5314 - acc: 0.771 - ETA: 9:09 - loss: 0.5160 - acc: 0.7

307/307 [==============================] - ETA: 3:21 - loss: 0.4650 - acc: 0.790 - ETA: 3:19 - loss: 0.4678 - acc: 0.790 - ETA: 3:17 - loss: 0.4657 - acc: 0.791 - ETA: 3:15 - loss: 0.4645 - acc: 0.792 - ETA: 3:13 - loss: 0.4633 - acc: 0.793 - ETA: 3:12 - loss: 0.4640 - acc: 0.792 - ETA: 3:10 - loss: 0.4637 - acc: 0.792 - ETA: 3:08 - loss: 0.4631 - acc: 0.791 - ETA: 3:06 - loss: 0.4629 - acc: 0.791 - ETA: 3:04 - loss: 0.4615 - acc: 0.792 - ETA: 3:02 - loss: 0.4609 - acc: 0.793 - ETA: 3:00 - loss: 0.4604 - acc: 0.793 - ETA: 2:58 - loss: 0.4603 - acc: 0.793 - ETA: 2:56 - loss: 0.4591 - acc: 0.793 - ETA: 2:54 - loss: 0.4585 - acc: 0.793 - ETA: 2:52 - loss: 0.4574 - acc: 0.793 - ETA: 2:50 - loss: 0.4574 - acc: 0.794 - ETA: 2:48 - loss: 0.4556 - acc: 0.795 - ETA: 2:46 - loss: 0.4548 - acc: 0.795 - ETA: 2:44 - loss: 0.4540 - acc: 0.795 - ETA: 2:42 - loss: 0.4532 - acc: 0.796 - ETA: 2:40 - loss: 0.4538 - acc: 0.795 - ETA: 2:38 - loss: 0.4559 - acc: 0.795 - ETA: 2:36 - loss: 0.4557 - acc: 0.795

204/307 [==================>...........] - ETA: 10:02 - loss: 0.4190 - acc: 0.75 - ETA: 9:58 - loss: 0.5824 - acc: 0.6250 - ETA: 7:37 - loss: 0.6114 - acc: 0.750 - ETA: 8:07 - loss: 0.4896 - acc: 0.812 - ETA: 8:26 - loss: 0.5561 - acc: 0.775 - ETA: 8:38 - loss: 0.5653 - acc: 0.770 - ETA: 8:44 - loss: 0.5893 - acc: 0.732 - ETA: 8:49 - loss: 0.5683 - acc: 0.750 - ETA: 8:58 - loss: 0.5259 - acc: 0.777 - ETA: 9:06 - loss: 0.5231 - acc: 0.775 - ETA: 9:08 - loss: 0.5007 - acc: 0.784 - ETA: 9:09 - loss: 0.4983 - acc: 0.770 - ETA: 9:10 - loss: 0.4820 - acc: 0.788 - ETA: 9:10 - loss: 0.4582 - acc: 0.803 - ETA: 9:09 - loss: 0.4425 - acc: 0.816 - ETA: 9:09 - loss: 0.4270 - acc: 0.820 - ETA: 9:09 - loss: 0.4162 - acc: 0.823 - ETA: 9:09 - loss: 0.4076 - acc: 0.826 - ETA: 9:08 - loss: 0.3939 - acc: 0.835 - ETA: 9:07 - loss: 0.3799 - acc: 0.843 - ETA: 9:07 - loss: 0.3762 - acc: 0.839 - ETA: 8:48 - loss: 0.3599 - acc: 0.846 - ETA: 8:48 - loss: 0.3472 - acc: 0.853 - ETA: 8:47 - loss: 0.3473 - acc: 0.84

307/307 [==============================] - ETA: 3:19 - loss: 0.3144 - acc: 0.892 - ETA: 3:17 - loss: 0.3164 - acc: 0.892 - ETA: 3:16 - loss: 0.3153 - acc: 0.892 - ETA: 3:14 - loss: 0.3159 - acc: 0.892 - ETA: 3:12 - loss: 0.3192 - acc: 0.890 - ETA: 3:10 - loss: 0.3197 - acc: 0.889 - ETA: 3:08 - loss: 0.3187 - acc: 0.890 - ETA: 3:06 - loss: 0.3182 - acc: 0.890 - ETA: 3:04 - loss: 0.3200 - acc: 0.890 - ETA: 3:02 - loss: 0.3188 - acc: 0.890 - ETA: 3:00 - loss: 0.3181 - acc: 0.890 - ETA: 2:58 - loss: 0.3170 - acc: 0.891 - ETA: 2:56 - loss: 0.3188 - acc: 0.890 - ETA: 2:54 - loss: 0.3183 - acc: 0.891 - ETA: 2:52 - loss: 0.3198 - acc: 0.889 - ETA: 2:50 - loss: 0.3197 - acc: 0.889 - ETA: 2:48 - loss: 0.3196 - acc: 0.889 - ETA: 2:46 - loss: 0.3188 - acc: 0.890 - ETA: 2:44 - loss: 0.3177 - acc: 0.890 - ETA: 2:42 - loss: 0.3179 - acc: 0.890 - ETA: 2:40 - loss: 0.3166 - acc: 0.891 - ETA: 2:38 - loss: 0.3173 - acc: 0.890 - ETA: 2:36 - loss: 0.3161 - acc: 0.891 - ETA: 2:34 - loss: 0.3153 - acc: 0.891

In [20]:
cmod = load_model('best_weight3_mod.hdf5') # model3 is performing well on test

In [22]:
#import glob
#cmod = load_model('best_weight4.hdf5')
tot = 0
cor = 0
for cl in range(3):
    for fn in glob.glob('test/' + str(cl) + '/*'):
        a_p = cv2.imread(fn)
        a_p = cv2.resize(a_p,(224,224))
        a_p = np.ndarray.astype(a_p,dtype='float32')
        a_p /= 255
        a_p = np.expand_dims(a_p,0)
        pred = cmod.predict_classes(a_p)
        if(cl == pred[0]):
            cor+=1
        elif(cl == 0 and pred[0] == 1):
            cor+=1
        elif(cl == 1 and pred[0] == 0):
            cor+=1
        tot += 1
        #print(pred)
print(cor/tot)

0.6


In [26]:
## model3 best so far, 2 class 83.3%, 2 class 66.6% 
## model3 modified 3 class 53.3, 2 class 73.3

In [4]:
from keras.initializers import he_normal
from keras import regularizers

In [15]:
## model 3 modified
model3 = Sequential()

model3.add(Conv2D(16, kernel_size=(5, 5),   # kernel size :
                 input_shape=(224,224,3)))
model3.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1))) # strides up :    strides down :
#model3.add(BatchNormalization()) # BN on :      BN off : 
model3.add(Activation('relu')) # sigmoid :      relu :      tanh : 

# LeakyReLU(alpha=0.1)
#model3.add(Dropout(0.01))

model3.add(Conv2D(32, (5, 5)))     # kernel size :
model3.add(MaxPooling2D(pool_size=(2, 2)))

#model3.add(BatchNormalization()) # BN on :      BN off : 
model3.add(Activation('relu')) # sigmoid :      relu :      tanh :
#model3.add(Dropout(0.02))

        
model3.add(Flatten())
model3.add(Dense(156))
#model3.add(BatchNormalization())
model3.add(Activation('relu'))
model3.add(Dropout(0.25))

#model3.add(Dense(64))
#model3.add(Activation('relu'))

#model3.add(Dense(32))
#model3.add(Activation('relu'))

#model3.add(Dense(16))
#model3.add(Activation('sigmoid'))

           
model3.add(Dense(3))
#model2.add(BatchNormalization())
model3.add(Activation('softmax'))

In [16]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 219, 219, 16)      0         
_________________________________________________________________
activation_9 (Activation)    (None, 219, 219, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 215, 215, 32)      12832     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 107, 107, 32)      0         
_________________________________________________________________
activation_10 (Activation)   (None, 107, 107, 32)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 366368)            0         
__________

In [17]:
TRAINING_DIR = 'train'
IMAGE_SIZE = 224
BATCH_SIZE = 5

In [18]:
optm = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model3.compile(loss=categorical_crossentropy,
              optimizer=optm,
              metrics=['accuracy'])
# checkpoint
path = "best_weight3_mod.hdf5"
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]



data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=19,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=True, seed=21,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")




Found 2457 images belonging to 3 classes.
Found 433 images belonging to 3 classes.


In [19]:
# 80, 80, 40
c_weight = {0:1, 1:1, 2:2}

model3.fit_generator(train_generator,
                       steps_per_epoch=2168//16,
                       validation_data=validation_generator, 
                       epochs=5, 
                       verbose=1,
                       class_weight = c_weight,
                       callbacks = callbacks_list,
                       validation_steps=722//16,
                       initial_epoch=0)

Epoch 1/5
135/135 [==============================] - 532s 4s/step - loss: 3.4944 - acc: 0.4119 - val_loss: 0.9946 - val_acc: 0.4844

Epoch 00001: val_acc improved from -inf to 0.48444, saving model to best_weight3_mod.hdf5
Epoch 2/5
135/135 [==============================] - 481s 4s/step - loss: 1.2343 - acc: 0.5215 - val_loss: 0.8453 - val_acc: 0.5733

Epoch 00002: val_acc improved from 0.48444 to 0.57333, saving model to best_weight3_mod.hdf5
Epoch 3/5
135/135 [==============================] - 453s 3s/step - loss: 1.0280 - acc: 0.5615 - val_loss: 0.8699 - val_acc: 0.5244

Epoch 00003: val_acc did not improve
Epoch 4/5
135/135 [==============================] - 464s 3s/step - loss: 0.8694 - acc: 0.5763 - val_loss: 0.8535 - val_acc: 0.4889

Epoch 00004: val_acc did not improve
Epoch 5/5
135/135 [==============================] - 439s 3s/step - loss: 0.7693 - acc: 0.6489 - val_loss: 0.7698 - val_acc: 0.5200

Epoch 00005: val_acc did not improve


In [6]:
resnet = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3), pooling='max')

In [7]:
for i, layers in enumerate(resnet.layers):
    print(i, layers.name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 activatio

**Reference:**
 1. https://datascience.stackexchange.com/questions/25122/pre-trained-cnn-for-feature-extraction
 2. https://keras.io/applications/
 3. https://arxiv.org/pdf/1805.08974.pdf {Do Better ImageNet Models Transfer Better?}
 4. https://github.com/johannesu/NASNet-keras
